In [ ]:
import json
import sys
import threading
import websocket
import simplejson
from time import sleep
from datetime import datetime
import logging
import os
from RofexAPI import PJS_RESTv3 as pjs
sys.path.append('C:/Users/pauli/Documents/python/OptionModelsV3/RofexAPI')


In [2]:
logging.basicConfig()

In [3]:
#test funcion datetime
year = str(datetime.today().year)
day = str(datetime.today().day)
m=0
m+=1
if(len(day)==1):
    day = "0"+day
month = str(datetime.today().month)
if(len(month)==1):
    month = "0"+month
print(year)
print(month)
print(day)
print (m)

2018
11
12
1


In [4]:
listaMensajes=[]
subsMDEvents = []
subsOREvents = []
mensajes=0
MSG_MDSuscription = '{{"type":"smd","level":1,"entries":[{e}],"products":[{s}]}}'
MSG_OSSuscription = '{{"type":"os","account":{{"id":"{a}"}}}}'
#MSG_Symbol = '{{"symbol":"{s}","marketId":"ROFX"}}'
MSG_Double_Quotes = '"{s}"'
#global mensajes

In [5]:
def on_message(ws, message):
   # now  = datetime.now()
        
    try:
        print ("en el try")
        # Valido Mensaje entrante
        msg = simplejson.loads(message)
        msgType = msg['type'].upper()


        if msgType == 'MD':
            # Aca es Market Data, esta escuchando novedades
            
            #subsMDEvents.append(msg)
            print("----------------------------------------------")
            #print("Mensajes:---->", len(subsMDEvents))
            global mensajes
            mensajes+=1
            print("Mensajes:---->", mensajes)
            
            
            print ("En mensaje MD")
            #sys.stdout.flush()
            now  = datetime.now()
            print(str(now), msg)
           
                      
            
            #print("before processinf msg:...")
            symbol=msg['instrumentId']['symbol']
            
            #print (symbol)
            
            #Aca hay un problema si no hay bid u offer pq solo viene ['marketData']
            bidMsg=msg['marketData']['BI']
            offerMsg=msg['marketData']['OF']
            #print("BID msg :", bidMsg)
            #print("OFFER msg :",offerMsg)
            if bidMsg==[]:
                print("------------------------------------------>No BID detected")
                bid=0
                bidSize=0
            else:
                bid=msg['marketData']['BI'][0]['price']
                bidSize=msg['marketData']['BI'][0]['size']
            if offerMsg==[]:
                print("------------------------------------------>No OFFER detected")
                offer=0
                offerSize=0
            else:
                offer=msg['marketData']['OF'][0]['price']
                offerSize=msg['marketData']['OF'][0]['size']
                       
            
            msgArray=[]          
            msgArray.append(symbol)
           
            msgArray.append(bid)
            msgArray.append(bidSize)
            msgArray.append(offer)
            msgArray.append(offerSize)
            listaMensajes.append(msgArray)
           
            
            print(symbol,"----->", bid,"/", offer," SIZE--> ",bidSize,"/",offerSize)
            print("------------------------------------------------------")
            
                       
            
            if (bidSize ==7):
                print("BID Size of ", symbol,"----------------->",bidSize)
            
                #mensajes=0
                
            
            #print("---> Instrument Id",msg['instrumentId']['symbol'])
            #print("---> Market Data", msg['marketData'])
            #print("---> Market  Data BID ",msg['marketData']['BI'][0]['price'])
            #print("---> Market Data Offer",msg['marketData']['OF'])
            #  TODO, aca va todo lo que se quiere hacer con los msg recibidos
            # 
        
            # data = msg;

            #si queremos escribir a un archivo usamos algo asi:
            #filename.write(str(now)+"|"+str(data['marketData']['LA']['price'])+"|"+str(data['marketData']['LA']['size'])+"|"+str(data['marketData']['BI'][0]['price'])+"|"+str(data['marketData']['BI'][0]['size'])+"|"+str(data['marketData']['OF'][0]['price'])+"|"+str(data['marketData']['OF'][0]['size'])+"\n");
            #filename.close()


        elif msgType == 'OR':
            print ("En Mensaje OR")
            print(msg)
        else:
            print ("Tipo de Mensaje Recibido No soportado: " + msg)
    except:
        print ("Error al procesar mensaje recibido: " + message)

def on_error(ws, error):
    print ("Salio por error")
    print (error)
    ws.close()
    
def on_close(ws):
    print ("### connection closed ###")
    
def on_open(ws):    
    
    print ("On Conection Open...")
    

In [ ]:
def initWS():
    # 1st set urls
    # 2nd user data 
    entorno=1
    pjs.setURLs()
    pjs.init("balanz","balanz","555",entorno)
    
    #parametros contratos a suscribir
    type_="smd"
    level_="1"
    marketId_="ROFX"
       
    #symbols=[]
    #symbols.append("RFX20Dic18")
    #symbols.append("DoDic18")
    symbols=pjs.suscribirContratos()
    print("Contratos suscriptos:")
    for s in symbols:
        print(s)
    print('\n')
    
    
    isLogin = pjs.login()
    if isLogin:
        
        # control
        print("Conexion establecida...")
        print('Token:'+' '+pjs.token)
        # end control
        
        headers = {'X-Auth-Token:{token}'.format(token=pjs.token)}

        ws = websocket.WebSocketApp(pjs.activeWSEndpoint,
                                         on_message=on_message,
                                         on_error=on_error,
                                         on_close=on_close,
                                         on_open=on_open,
                                         header=headers)

        wst = threading.Thread(target=ws.run_forever, kwargs={"ping_interval" :5})
        
        wst.start()

        # Esperamos a que la conexion ws se establezca
        conn_timeout = 5
        #conn_timeout = 50 #y nada
        sleep(1)


        while not ws.sock.connected and conn_timeout:
            sleep(1)
            conn_timeout -= 1
        else:
            
        
            for sym in symbols:
                # Aca se definen los prod a suscribir
                # tienen la forma:
                # aaa= "{\"type\":\"smd\",\"level\":1, \"entries\":[\"BI\", \"OF\"],\"products\":[{\"symbol\":\"DODic18\",\"marketId\":\"ROFX\"}]}"
                
                aaa = "{\"type\":\""+type_+"\",\"level\":"+level_+", \"entries\":[\"BI\", \"OF\"],\"products\":[{\"symbol\":\""+sym+"\",\"marketId\":\""+marketId_+"\"}]}"
                
                ws.send(aaa)
                #ws.send(bbb)
                                    
                #mensajes enviados
                print("Sent")
                print(aaa)
                #print(bbb)
            
                print("Receiving...")
                
                sleep(1) #y nada

    else:
        print( "Error al autenticarse...")


if __name__ == "__main__":
    initWS()
    #ws = {"type":"smd","level":1, "entries":["BI", "OF"],"products":[{"symbol":"DOJul18","marketId":"ROFX"}]};
    #on_newmsg_ale()

    #on_message(ws, message):


    #if(ws.sock.connected):

setURLs Ok...
init() Ok...
login() OK...
Conexion establecida...
Token: H/lVZY0sFcYMISEgol6M0mxVbGtDjZdxeWW7M+g8k9Q=
On Conection Open...
Sent
{"type":"smd","level":1, "entries":["BI", "OF"],"products":[{"symbol":"RFX20Dic18","marketId":"ROFX"}]}
Receiving...
en el try
----------------------------------------------
Mensajes:----> 1
En mensaje MD
2018-11-12 13:26:10.988211 {'type': 'Md', 'timestamp': 1542039963287, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'RFX20Dic18'}, 'marketData': {'BI': [{'price': 39272, 'size': 32}], 'OF': [{'price': 39285, 'size': 6}]}}
RFX20Dic18 -----> 39272 / 39285  SIZE-->  32 / 6
------------------------------------------------------
Sent
{"type":"smd","level":1, "entries":["BI", "OF"],"products":[{"symbol":"DoDic18","marketId":"ROFX"}]}
Receiving...
en el try
----------------------------------------------
Mensajes:----> 2
En mensaje MD
2018-11-12 13:26:11.986311 {'type': 'Md', 'timestamp': 1542039964286, 'instrumentId': {'marketId': 'ROFX', 'symbol': '

en el try
----------------------------------------------
Mensajes:----> 20
En mensaje MD
2018-11-12 13:28:16.529648 {'type': 'Md', 'timestamp': 1542040088831, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'DODic18'}, 'marketData': {'BI': [{'price': 38.08, 'size': 97}], 'OF': [{'price': 38.09, 'size': 3}]}}
DODic18 -----> 38.08 / 38.09  SIZE-->  97 / 3
------------------------------------------------------
en el try
----------------------------------------------
Mensajes:----> 21
En mensaje MD
2018-11-12 13:28:17.029648 {'type': 'Md', 'timestamp': 1542040089331, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'DODic18'}, 'marketData': {'BI': [{'price': 38.08, 'size': 97}], 'OF': [{'price': 38.092, 'size': 553}]}}
DODic18 -----> 38.08 / 38.092  SIZE-->  97 / 553
------------------------------------------------------
en el try
----------------------------------------------
Mensajes:----> 22
En mensaje MD
2018-11-12 13:28:24.473648 {'type': 'Md', 'timestamp': 1542040096775, 'instrumentId':

en el try
----------------------------------------------
Mensajes:----> 40
En mensaje MD
2018-11-12 13:29:39.033648 {'type': 'Md', 'timestamp': 1542040171334, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'RFX20Dic18'}, 'marketData': {'BI': [{'price': 39206, 'size': 1}], 'OF': [{'price': 39235, 'size': 1}]}}
RFX20Dic18 -----> 39206 / 39235  SIZE-->  1 / 1
------------------------------------------------------
en el try
----------------------------------------------
Mensajes:----> 41
En mensaje MD
2018-11-12 13:29:42.035648 {'type': 'Md', 'timestamp': 1542040174343, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'RFX20Dic18'}, 'marketData': {'BI': [{'price': 39202, 'size': 3}], 'OF': [{'price': 39235, 'size': 1}]}}
RFX20Dic18 -----> 39202 / 39235  SIZE-->  3 / 1
------------------------------------------------------
en el try
----------------------------------------------
Mensajes:----> 42
En mensaje MD
2018-11-12 13:29:51.803648 {'type': 'Md', 'timestamp': 1542040184111, 'instrumentId